In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
sql_context = SparkSession.builder.master('local[*]').getOrCreate()
sql_context

23/03/17 15:31:16 WARN Utils: Your hostname, siri-HP-Notebook resolves to a loopback address: 127.0.1.1; using 192.168.1.7 instead (on interface wlo1)
23/03/17 15:31:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/17 15:31:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/17 15:31:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
  data = sql_context.read.csv("./sangam.csv",header=True,inferSchema='True')
  data.show(3)

+-------------------+-----------------+------------------+----------+---------+---------+-----------------+---------+
|               Date|               DO|                pH|       ORP|     Cond|     Temp|              WQI|   Status|
+-------------------+-----------------+------------------+----------+---------+---------+-----------------+---------+
|2019-01-12 15:33:16|9.494212037759977|13.765933596525262|0.14840198|12.954404|17.830261|54.81198760710678|Very Poor|
|2019-01-12 15:34:17|9.500406233111164|13.337534775077296| 0.1445036| 8.547796|17.798553|51.48805043895461|Very Poor|
|2019-01-12 15:35:18|9.487447743811652|13.198463167914054|0.13437152|16.847918| 17.86493|50.42070179995088|Very Poor|
+-------------------+-----------------+------------------+----------+---------+---------+-----------------+---------+
only showing top 3 rows



In [5]:
import os

import csv
import itertools
import collections

from scipy import stats

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

In [6]:
data=data.drop('Date')
data

DataFrame[DO: double, pH: double, ORP: double, Cond: double, Temp: double, WQI: double, Status: string]

In [7]:
from pyspark.ml.feature import StringIndexer

In [8]:
ind = StringIndexer(inputCol = 'Status', outputCol = 'Status_index')
data=ind.fit(data).transform(data)

In [9]:
from pyspark.ml.feature import VectorAssembler

numericCols = ['DO', 'pH', 'ORP', 'Cond','Temp','WQI']
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")

In [10]:
data = assembler.transform(data)
data.show(5)

+-----------------+------------------+----------+---------+---------+-----------------+---------+------------+--------------------+
|               DO|                pH|       ORP|     Cond|     Temp|              WQI|   Status|Status_index|            features|
+-----------------+------------------+----------+---------+---------+-----------------+---------+------------+--------------------+
|9.494212037759977|13.765933596525262|0.14840198|12.954404|17.830261|54.81198760710678|Very Poor|         2.0|[9.49421203775997...|
|9.500406233111164|13.337534775077296| 0.1445036| 8.547796|17.798553|51.48805043895461|Very Poor|         2.0|[9.50040623311116...|
|9.487447743811652|13.198463167914054|0.13437152|16.847918| 17.86493|50.42070179995088|Very Poor|         2.0|[9.48744774381165...|
|9.486121036332559|12.732116142804621|0.14270854|16.884756|17.871735|  46.901646035597|Very Poor|         2.0|[9.48612103633255...|
|9.485210958535616| 13.28446675206912|0.13752413|16.987082|17.876404|51.1046

In [11]:
final=data.select("features","Status_index")
final.show()

+--------------------+------------+
|            features|Status_index|
+--------------------+------------+
|[9.49421203775997...|         2.0|
|[9.50040623311116...|         2.0|
|[9.48744774381165...|         2.0|
|[9.48612103633255...|         2.0|
|[9.48521095853561...|         2.0|
|[18.413895,13.107...|         2.0|
|[17.560917,13.272...|         2.0|
|[17.419922,13.380...|         2.0|
|[17.972828,13.549...|         2.0|
|[9.48054395114254...|         2.0|
|[9.47934358211962...|         2.0|
|[16.906841,13.543...|         2.0|
|[16.089485,13.763...|         2.0|
|[17.893639,13.068...|         2.0|
|[10.478089,12.935...|         2.0|
|[18.264702,13.677...|         2.0|
|[18.571953,13.308...|         2.0|
|[17.599611,13.232...|         2.0|
|[11.472302,13.474...|         2.0|
|[9.42847816151952...|         2.0|
+--------------------+------------+
only showing top 20 rows



In [12]:
train, test = final.randomSplit([0.6, 0.4], seed = 2018)

In [13]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier

In [14]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'Status_index',maxIter=50, regParam=0.01, elasticNetParam=0.0)
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'Status_index')
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'Status_index',maxDepth=5)

In [15]:
lrmodel = lr.fit(train)

In [16]:
rfModel = rf.fit(train)

In [17]:
dtModel = dt.fit(train)

In [18]:
lrmodel.save("lr_model")
rfModel.save("rf_model")
dtModel.save("dt__model")



Py4JJavaError: An error occurred while calling o365.save.
: java.io.IOException: Path lr_model already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at p

In [ ]:
import pickle

pickle.dump(lrmodel, open('lr.pkl', 'wb'))

TypeError: cannot pickle '_thread.RLock' object

In [ ]:
lrPrediction = lrmodel.transform(test)
lrPrediction.show(3)

: 

In [ ]:
dtPrediction = dtModel.transform(test)

: 

In [ ]:
rfPrediction = rfModel.transform(test)

: 

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="Status_index", predictionCol="prediction")
accuracy = evaluator.evaluate(lrPrediction)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

: 

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="Status_index", predictionCol="prediction")
accuracy = evaluator.evaluate(dtPrediction)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

: 

In [ ]:
evaluator = MulticlassClassificationEvaluator(labelCol="Status_index", predictionCol="prediction")
accuracy = evaluator.evaluate(rfPrediction)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))

: 

In [ ]:
from sklearn.metrics import confusion_matrix
y_pred= lrPrediction.select('prediction').collect()
y_orig= lrPrediction.select('Status_index').collect()
cm = confusion_matrix(y_orig,y_pred)
print(cm)

: 

In [ ]:
plt.figure(figsize=(7,5))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = "Accuracy Score: {0}".format(accuracy)
plt.title(all_sample_title, size = 15);

: 

: 

: 

: 

: 

: 